# Практическое задание №5 по теме "Part-of-Speech разметка, NER, извлечение отношений".

Загрузка библиотек:

In [1]:
!pip install pyconll

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [33]:
!pip install corus

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.5/83.5 kB 6.8 MB/s eta 0:00:00


In [38]:
!pip install razdel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [41]:
import nltk
import pandas as pd
from nltk.tokenize import word_tokenize
import matplotlib
from nltk.corpus import brown
from nltk.tag import UnigramTagger, BigramTagger, TrigramTagger
import pyconll
import corus

%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [39]:
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from razdel import tokenize

Загрузка и разбиение данных:

In [3]:
!wget -O ./ru_syntagrus-ud-train-b.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-b.conllu
!wget -O ./ru_syntagrus-ud-dev.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu

--2023-04-17 13:49:01--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-b.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42819832 (41M) [text/plain]
Saving to: ‘./ru_syntagrus-ud-train-b.conllu’

./ru_syntagrus-ud-t 100%[===================>]  40.84M   244MB/s    in 0.2s    

2023-04-17 13:49:01 (244 MB/s) - ‘./ru_syntagrus-ud-train-b.conllu’ saved [42819832/42819832]

--2023-04-17 13:49:01--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185

In [4]:
full_train = pyconll.load_from_file('ru_syntagrus-ud-train-b.conllu')
full_test = pyconll.load_from_file('ru_syntagrus-ud-dev.conllu')

In [5]:
for sent in full_train[:2]:
    for token in sent:
        print(token.form, token.upos)
    print()

Премьер-министр NOUN
РФ PROPN
Владимир PROPN
Путин PROPN
подписал VERB
распоряжение NOUN
о ADP
переводе NOUN
Государственного ADJ
университета NOUN
- PUNCT
Высшей ADJ
школы NOUN
экономики NOUN
( PUNCT
ГУ-ВШЭ PROPN
) PUNCT
из ADP
ведения NOUN
Минэкономразвития PROPN
в ADP
ведение NOUN
Правительства PROPN
РФ PROPN
. PUNCT

По ADP
словам NOUN
первого ADJ
проректора NOUN
ГУ-ВШЭ PROPN
Льва PROPN
Якобсона PROPN
, PUNCT
этот DET
вопрос NOUN
обсуждался VERB
давно ADV
. PUNCT



In [6]:
fdata_train = []
for sent in full_train[:]:
    fdata_train.append([(token.form, token.upos) for token in sent])
    
fdata_test = []
for sent in full_test[:]:
    fdata_test.append([(token.form, token.upos) for token in sent])
    
fdata_sent_test = []
for sent in full_test[:]:
    fdata_sent_test.append([token.form for token in sent])

Запуск Теггеров по отдельности и в комбинации:

In [9]:
unigram_tagger = UnigramTagger(fdata_train)
display(unigram_tagger.tag(fdata_sent_test[100]))

[('Это', 'PRON'),
 ('сочинение', 'NOUN'),
 ('известно', 'ADJ'),
 ('во', 'ADP'),
 ('многих', 'NUM'),
 ('вариантах', 'NOUN'),
 ('(', 'PUNCT'),
 ('самые', 'ADJ'),
 ('ранние', 'ADJ'),
 ('из', 'ADP'),
 ('них', 'PRON'),
 ('почти', 'ADV'),
 ('на', 'ADP'),
 ('сто', 'NUM'),
 ('лет', 'NOUN'),
 ('старше', 'ADJ'),
 (')', 'PUNCT'),
 ('и', 'CCONJ'),
 ('восходит', 'VERB'),
 ('к', 'ADP'),
 ('ещё', 'ADV'),
 ('более', 'ADV'),
 ('древним', None),
 ('рукописям', None),
 ('XVI', 'ADJ'),
 ('в', 'ADP'),
 ('.', 'PUNCT')]

In [10]:
bigram_tagger = BigramTagger(fdata_train, backoff=unigram_tagger)
display(bigram_tagger.tag(fdata_sent_test[100]))

[('Это', 'PRON'),
 ('сочинение', 'NOUN'),
 ('известно', 'ADJ'),
 ('во', 'ADP'),
 ('многих', 'NUM'),
 ('вариантах', 'NOUN'),
 ('(', 'PUNCT'),
 ('самые', 'ADJ'),
 ('ранние', 'ADJ'),
 ('из', 'ADP'),
 ('них', 'PRON'),
 ('почти', 'ADV'),
 ('на', 'ADP'),
 ('сто', 'NUM'),
 ('лет', 'NOUN'),
 ('старше', 'ADJ'),
 (')', 'PUNCT'),
 ('и', 'CCONJ'),
 ('восходит', 'VERB'),
 ('к', 'ADP'),
 ('ещё', 'ADV'),
 ('более', 'ADV'),
 ('древним', None),
 ('рукописям', None),
 ('XVI', 'ADJ'),
 ('в', 'ADP'),
 ('.', 'PUNCT')]

In [11]:
trigram_tagger = TrigramTagger(fdata_train, backoff=bigram_tagger)
display(trigram_tagger.tag(fdata_sent_test[100]))

[('Это', 'PRON'),
 ('сочинение', 'NOUN'),
 ('известно', 'ADJ'),
 ('во', 'ADP'),
 ('многих', 'NUM'),
 ('вариантах', 'NOUN'),
 ('(', 'PUNCT'),
 ('самые', 'ADJ'),
 ('ранние', 'ADJ'),
 ('из', 'ADP'),
 ('них', 'PRON'),
 ('почти', 'ADV'),
 ('на', 'ADP'),
 ('сто', 'NUM'),
 ('лет', 'NOUN'),
 ('старше', 'ADJ'),
 (')', 'PUNCT'),
 ('и', 'CCONJ'),
 ('восходит', 'VERB'),
 ('к', 'ADP'),
 ('ещё', 'ADV'),
 ('более', 'ADV'),
 ('древним', None),
 ('рукописям', None),
 ('XVI', 'ADJ'),
 ('в', 'ADP'),
 ('.', 'PUNCT')]

In [12]:
def backoff_tagger(train_sents, tagger_classes, backoff=None):
    for cls in tagger_classes:
        backoff = cls(train_sents, backoff=backoff)
    return backoff

backoff = DefaultTagger('NN') 
tag = backoff_tagger(fdata_train, [UnigramTagger, BigramTagger, TrigramTagger], backoff = backoff) 

Результаты:

In [16]:
print(f'unigram_tagger: {unigram_tagger.evaluate(fdata_test)}')
print(f'bigram_tagger: {bigram_tagger.evaluate(fdata_test)}')
print(f'trigram_tagger: {trigram_tagger.evaluate(fdata_test)}')
print(f'backoff_tagger: {tag.evaluate(fdata_test)}')

unigram_tagger: 0.8317338368383358
bigram_tagger: 0.8379842437658702
trigram_tagger: 0.8367016081776157
backoff_tagger: 0.836792759945309


**Вывод:** Хотя и все теггеры показали примерно одинаковый результат, лидирует биграмный теггер, обойдя даже комбинацию теггеров. Это указывает на то, что вероятно результат во много м зависит от самих данных, нежели от метода.

Теггер с векторайзерами:

In [18]:
train_tok = []
train_label = []
for sent in fdata_train[:]:
    for tok in sent:
        train_tok.append(tok[0])
        train_label.append('NO_TAG' if tok[1] is None else tok[1])
        
test_tok = []
test_label = []
for sent in fdata_test[:]:
    for tok in sent:
        test_tok.append(tok[0])
        test_label.append('NO_TAG' if tok[1] is None else tok[1])

In [19]:
le = LabelEncoder()
train_enc_labels = le.fit_transform(train_label)

In [20]:
test_enc_labels = le.transform(test_label)

In [23]:
hvectorizer = HashingVectorizer(ngram_range=(1, 3), analyzer='char', n_features=50)

In [26]:
X_train = hvectorizer.fit_transform(train_tok)
X_test = hvectorizer.transform(test_tok)

AttributeError: ignored

### Данную ошибку исправить не смог. Видимо что-то с данными...

In [ ]:
lr = LogisticRegression(random_state=0, max_iter=10)
lr.fit(X_train, train_enc_labels)

In [ ]:
pred = lr.predict(X_test)
accuracy_score(test_enc_labels, pred)

Проверяем NER:

In [35]:
!wget http://www.labinform.ru/pub/named_entities/collection5.zip

--2023-04-17 14:24:16--  http://www.labinform.ru/pub/named_entities/collection5.zip
Resolving www.labinform.ru (www.labinform.ru)... 95.181.230.181
Connecting to www.labinform.ru (www.labinform.ru)|95.181.230.181|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1899530 (1.8M) [application/zip]
Saving to: ‘collection5.zip’

collection5.zip     100%[===================>]   1.81M  5.63MB/s    in 0.3s    

2023-04-17 14:24:17 (5.63 MB/s) - ‘collection5.zip’ saved [1899530/1899530]



In [36]:
!unzip collection5.zip

Archive:  collection5.zip
   creating: Collection5/
  inflating: Collection5/001.ann     
  inflating: Collection5/001.txt     
  inflating: Collection5/002.ann     
  inflating: Collection5/002.txt     
  inflating: Collection5/003.ann     
  inflating: Collection5/003.txt     
  inflating: Collection5/004.ann     
  inflating: Collection5/004.txt     
  inflating: Collection5/005.ann     
  inflating: Collection5/005.txt     
  inflating: Collection5/006.ann     
  inflating: Collection5/006.txt     
  inflating: Collection5/007.ann     
  inflating: Collection5/007.txt     
  inflating: Collection5/008.ann     
  inflating: Collection5/008.txt     
  inflating: Collection5/009.ann     
  inflating: Collection5/009.txt     
  inflating: Collection5/010.ann     
  inflating: Collection5/010.txt     
  inflating: Collection5/011.ann     
  inflating: Collection5/011.txt     
  inflating: Collection5/012.ann     
  inflating: Collection5/012.txt     
  inflating: Collection5/013.ann    

In [37]:
from corus import load_ne5

dir = 'Collection5/'
records = load_ne5(dir)
next(records)

Ne5Markup(
    id='303',
    text='Председателем совета директоров "Полюс Золота" переизбрана Е.Сальникова из ОНЭКСИМа.\r\n\r\nПредседателем совета директоров "Полюс Золото" переизбрана заместитель финансового директора группы "ОНЭКСИМ" Екатерина Сальникова. Как сообщили РБК в компании "Полюс Золото", такое решение было принято на заседании совета директоров.\r\n\r\nКак сообщалось ранее, в мае акционеры "Полюс Золота" на годовом собрании утвердили совет директоров в прежнем составе, в который вошли 9 человек: Е.Сальникова, президент группы "ОНЭКСИМ" Михаил Прохоров, генеральный директор, председатель совета директоров KazakhGold Group Limited Евгений Иванов, президент ГМК "Интергео" Максим Финский, управляющие директора представительства "Нафта Москва" - Павел Грачев и Анна Колончина, председатель совета директоров "Нафта Москва" Александр Мосионжик, независимый директор "Полюс Золота" Зумруд Рустамова и партнер-основатель Policy Partnership Limited Ltd. лорд Кланвиллиам (бывший лорд Г

In [40]:
words_docs = []
for ix, rec in enumerate(records):
    words = []
    for token in tokenize(rec.text):
        type_ent = 'OUT'
        for ent in rec.spans:
            if (token.start >= ent.start) and (token.stop <= ent.stop):
                type_ent = ent.type
                break
        words.append([token.text, type_ent])
    words_docs.extend(words)

In [46]:
df_words = pd.DataFrame(words_docs, columns=['word', 'tag'])
df_words.head(5)

,word,tag
0,США,GEOPOLIT
1,обеспокоены,OUT
2,беспорядками,OUT
3,в,OUT
4,Египте,GEOPOLIT


## Не получилось эти данные загрузить в NLTK.

Пришлось взять другие:

In [52]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [55]:
nltk.download('maxent_ne_chunker')

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.


True

In [57]:
nltk.download('words')

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [53]:
import requests
from bs4 import BeautifulSoup
import re

def url_to_string(url):
    res = requests.get(url)
    html = res.text
    soup = BeautifulSoup(html, 'html5lib')
    for script in soup(["script", "style", 'aside']):
        script.extract()
    return " ".join(re.split(r'[\n\t]+', soup.get_text()))

document = url_to_string('https://lenta.ru/news/2023/04/17/vasser/')

nltk.pos_tag(nltk.word_tokenize(document))

[('Вассерман', 'JJ'),
 ('призвал', 'NNP'),
 ('наказывать', 'NNP'),
 ('за', 'NNP'),
 ('тайное', 'NNP'),
 ('хранение', 'NNP'),
 ('наличных', 'NNP'),
 ('свыше', 'NNP'),
 ('миллиона', 'NNP'),
 ('рублей', 'NN'),
 (':', ':'),
 ('Капитал', 'NN'),
 (':', ':'),
 ('Экономика', 'NN'),
 (':', ':'),
 ('Lenta.ruГлавноеРоссияМирБывший', 'NNP'),
 ('СССРЭкономикаСиловые', 'NNP'),
 ('структурыНаука', 'NNP'),
 ('и', 'NNP'),
 ('техникаКультураСпортИнтернет', 'NNP'),
 ('и', 'NNP'),
 ('СМИЦенности', 'NNP'),
 ('ПутешествияИз', 'NNP'),
 ('жизниСреда', 'NNP'),
 ('обитанияЗабота', 'NNP'),
 ('о', 'NNP'),
 ('себеВойтиСтатьиГалереиВидеоСпецпроектыМоторАрхивЛента', 'NNP'),
 ('добраХочешь', 'NNP'),
 ('видеть', 'NNP'),
 ('только', 'NNP'),
 ('хорошие', 'NNP'),
 ('новости', 'NNP'),
 ('?', '.'),
 ('Жми', 'NN'),
 ('!', '.'),
 ('Вернуться', 'JJ'),
 ('в', 'JJ'),
 ('обычную', 'NN'),
 ('ленту', 'NN'),
 ('?', '.'),
 ('ВойтиЭкономикаВсеГосэкономикаБизнесРынкиКапиталДеловой', 'JJ'),
 ('климатСоциальная', 'NN'),
 ('сфераКриптова

In [58]:
{(' '.join(c[0] for c in chunk), chunk.label() ) for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(document))) if hasattr(chunk, 'label') }

{('Аксаков', 'PERSON'),
 ('Анатолий Вассерман', 'ORGANIZATION'),
 ('Бузовой', 'PERSON'),
 ('Вассерман', 'PERSON'),
 ('Госдумы', 'PERSON'),
 ('Госуслуги', 'PERSON'),
 ('Максим Блинов', 'PERSON'),
 ('Москвы', 'PERSON'),
 ('ООО', 'ORGANIZATION'),
 ('Однозначно', 'PERSON'),
 ('Софья ЕрмаковаОбсудитьПоследние', 'PERSON'),
 ('Только', 'PERSON'),
 ('Флориды', 'PERSON'),
 ('Шнуров', 'PERSON'),
 ('Юнайтед', 'PERSON'),
 ('климатСоциальная', 'ORGANIZATION')}

### Spacy:

In [60]:
!pip install -U spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 58.2 MB/s eta 0:00:00
  Attempting uninstall: spacy
    Found existing installation: spacy 3.5.1
    Uninstalling spacy-3.5.1:
      Successfully uninstalled spacy-3.5.1


In [61]:
!python -m spacy info

2023-04-17 14:56:11.624965: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT

============================== Info about spaCy ==============================

spaCy version    3.5.2                         
Location         /usr/local/lib/python3.9/dist-packages/spacy
Platform         Linux-5.10.147+-x86_64-with-glibc2.31
Python version   3.9.16                        
Pipelines        en_core_web_sm (3.5.0)        



In [63]:
!python -m spacy download ru_core_news_sm

2023-04-17 14:57:30.050566: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 40.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 61.1 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13721 sha256=ce5a17c22381711f80120b0a03abcdd9096b07a9cf3730ecf8f025551162a6dc
  Stored in directory: /root/.cache/pip/wheels/70/4a/46/1309fc853b8d395e60bafaf1b6df7845bdd82c95fd59dd8d2b
Successfully built docopt
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')


In [65]:
import spacy
from spacy import displacy

nlp = spacy.load("ru_core_news_sm")
document = url_to_string('https://lenta.ru/news/2023/04/17/vasser/')
article = nlp(document)
displacy.render(article, jupyter=True, style='ent')


### Самописный NER:

In [74]:
import tensorflow as tf

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, GlobalMaxPooling1D, Conv1D, GRU, LSTM, Dropout, Input
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [75]:
from sklearn import model_selection, preprocessing, linear_model

train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df_words['word'], df_words['tag'])

encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [76]:
train_data = tf.data.Dataset.from_tensor_slices((train_x, train_y))
valid_data = tf.data.Dataset.from_tensor_slices((valid_x, valid_y))

train_data = train_data.batch(16)
valid_data = valid_data.batch(16)

In [77]:
AUTOTUNE = tf.data.AUTOTUNE

train_data = train_data.cache().prefetch(buffer_size=AUTOTUNE)
valid_data = valid_data.cache().prefetch(buffer_size=AUTOTUNE)

In [78]:
def custom_standardization(input_data):
    return input_data

vocab_size = 30000
seq_len = 10

vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=seq_len)

text_data = train_data.map(lambda x, y: x)
vectorize_layer.adapt(text_data)

In [79]:
embedding_dim = 64

class modelNER(tf.keras.Model):
    def __init__(self):
        super(modelNER, self).__init__()
        self.emb = Embedding(vocab_size, embedding_dim)
        self.gPool = GlobalMaxPooling1D()
        self.fc1 = Dense(300, activation='relu')
        self.fc2 = Dense(50, activation='relu')
        self.fc3 = Dense(6, activation='softmax')

    def call(self, x):
        x = vectorize_layer(x)
        x = self.emb(x)
        pool_x = self.gPool(x)
        
        fc_x = self.fc1(pool_x)
        fc_x = self.fc2(fc_x)
        
        concat_x = tf.concat([pool_x, fc_x], axis=1)
        prob = self.fc3(concat_x)
        return prob

In [80]:
mmodel = modelNER()
mmodel.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])
mmodel.fit(train_data, validation_data=valid_data, epochs=3)

Epoch 1/3
12432/12432 [==============================] - 367s 29ms/step - loss: 0.2946 - accuracy: 0.9144 - val_loss: 0.2121 - val_accuracy: 0.9383
Epoch 2/3
12432/12432 [==============================] - 331s 27ms/step - loss: 0.1261 - accuracy: 0.9625 - val_loss: 0.3094 - val_accuracy: 0.8932
Epoch 3/3
12432/12432 [==============================] - 313s 25ms/step - loss: 0.1107 - accuracy: 0.9654 - val_loss: 0.2695 - val_accuracy: 0.8851


In [81]:
preds = mmodel.predict(valid_x)
labels_valid_y = encoder.inverse_transform(valid_y)

2072/2072 [==============================] - 5s 2ms/step


In [82]:
import numpy as np

In [83]:
preds_proba_max = [np.argmax(l) for l in preds]
preds_proba = encoder.inverse_transform(preds_proba_max)

In [84]:
from sklearn.metrics import classification_report

print(classification_report(labels_valid_y, preds_proba))

              precision    recall  f1-score   support

    GEOPOLIT       0.88      0.90      0.89      1111
         LOC       0.83      0.79      0.81      1170
       MEDIA       0.93      0.79      0.85       610
         ORG       0.29      0.66      0.40      3381
         OUT       0.97      0.92      0.94     54703
         PER       0.98      0.71      0.82      5326

    accuracy                           0.89     66301
   macro avg       0.81      0.79      0.79     66301
weighted avg       0.93      0.89      0.90     66301



Судя по accuracy, итоговый результат получился более чем хорошим.